In [ ]:
import os
!pip install tqdm retrowrapper gym-retro
os.chdir("/usr/local/lib/python3.6/dist-packages/")
!git clone https://github.com/openai/baselines.git
!git clone https://github.com/openai/retro-contest.git
os.chdir("/usr/local/lib/python3.6/dist-packages/baselines")
!pip install e .

os.chdir("/usr/local/lib/python3.6/dist-packages/retro-contest/support")
!pip install e .

     |████████████████████████████████| 162.0MB 92kB/s 
Cloning into 'baselines'...
remote: Enumerating objects: 3627, done.
remote: Total 3627 (delta 0), reused 0 (delta 0), pack-reused 3627
Receiving objects: 100% (3627/3627), 6.46 MiB | 15.79 MiB/s, done.
Resolving deltas: 100% (2429/2429), done.
Cloning into 'retro-contest'...
remote: Enumerating objects: 645, done.
remote: Total 645 (delta 0), reused 0 (delta 0), pack-reused 645
Receiving objects: 100% (645/645), 80.36 KiB | 4.23 MiB/s, done.
Resolving deltas: 100% (429/429), done.
Processing /usr/local/lib/python3.6/dist-packages/baselines
     |████████████████████████████████| 1.6MB 9.8MB/s 
  Created wheel for e: filename=e-1.4.5-cp37-none-any.whl size=2829 sha256=90ccb2b6ed45f7a327ab3c55ee36d1d317fe200f36c5e9fbf3216b11c4484af7
  Stored in directory: /root/.cache/pip/wheels/cb/db/09/e4af9f74b90354a841a09ea851687727f64ceeac6e554355d9
  Created wheel for baselines: filename=baselines-0.1.6-cp37-none-any.whl size=220664 sha256=5a

In [ ]:
import os
from google.colab import files

os.chdir("/content")
!ls
!mkdir -p data/roms/
print("upload roms")
os.chdir("data/roms/")
files.upload()
!ls
os.chdir("/content")
print("upload models")
files.upload()
!ls

In [ ]:
!python3 -m retro.import data/roms/

/usr/bin/python3: Error while finding module specification for 'retro.import' (ModuleNotFoundError: No module named 'retro')


In [ ]:
import retro
for game in retro.data.list_games():
    print(game, retro.data.list_states(game))

1942-Nes ['1Player.Level1']
1943-Nes ['Level1']
3NinjasKickBack-Genesis ['1Player.Colt.Level1']
8Eyes-Nes ['1Player.Arabia.Level1']
AaahhRealMonsters-Genesis ['Level1']
AbadoxTheDeadlyInnerWar-Nes ['Level1']
AcceleBrid-Snes ['SilverMarine.DefaultSettings.Level1']
ActRaiser2-Snes ['Level1']
ActionPachio-Snes ['Level1']
AddamsFamily-GameBoy ['Level1']
AddamsFamily-Genesis ['Level1']
AddamsFamily-Nes ['Level1']
AddamsFamily-Sms ['Level1']
AddamsFamily-Snes ['Level1']
AddamsFamilyPugsleysScavengerHunt-Nes ['Level1']
AddamsFamilyPugsleysScavengerHunt-Snes ['Level1']
AdvancedBusterhawkGleylancer-Genesis ['NormalMover.Stage1']
Adventure-Atari2600 ['Start']
AdventureIsland-GameBoy ['Level1']
AdventureIsland3-Nes ['Level1']
AdventureIslandII-Nes ['FernIsland.Level1']
AdventuresOfBatmanAndRobin-Genesis ['Level1.Batman']
AdventuresOfBayouBilly-Nes ['GameA.Level1']
AdventuresOfDinoRiki-Nes ['1Player.Level1']
AdventuresOfDrFranken-Snes ['Level1']
AdventuresOfKidKleets-Snes ['Level1.London']
Adventu

In [ ]:
%tensorflow_version 1.x
# Part taken from adborghi fantastic implementation
# https://github.com/aborghi/retro_contest_agent/blob/master/fastlearner/ppo2ttifrutti_sonic_env.py
import numpy as np
import gym
import os

#import gym_remote.client as grc
#import retrowrapper
#from retro_contest.local import make
from retro import make as make_retro

# This will be useful for stacking frames
from baselines.baselines.common.atari_wrappers import FrameStack

# Library used to modify frames (former times we used matplotlib)
import cv2

#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


# This function selects the probability distribution over actions
from baselines.common.distributions import make_pdtype

import time
#import os.path as osp

from baselines import logger

#import matplotlib.pyplot as plt

# Calculate cross entropy
from baselines.a2c.utils import cat_entropy

from baselines.common import explained_variance
from baselines.baselines.common.runners import AbstractEnvRunner
import math

# SubprocVecEnv creates a vector of n environments to run them simultaneously.
from baselines.common.vec_env.subproc_vec_env import SubprocVecEnv

In [ ]:
# setUseOpenCL = False means that we will not use GPU (disable OpenCL acceleration)
cv2.ocl.setUseOpenCL(True)

class PreprocessFrame(gym.ObservationWrapper):
  """
  Here we do the preprocessing part:
  - Set frame to gray
  - Resize the frame to 96x96x1
  """
  def __init__(self, env):
    gym.ObservationWrapper.__init__(self, env)
    self.width = 80
    self.height = 80
    self.observation_space = gym.spaces.Box(low=0, high=255, shape=(self.height, self.width, 1), dtype=np.uint8)

  def observation(self, frame):
    # Set frame to gray
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

    # Resize the frame to 96x96x1
    frame = cv2.resize(frame, (self.width, self.height), interpolation=cv2.INTER_AREA)
    frame = frame[:, :, None]

    return frame

class ActionsDiscretizer(gym.ActionWrapper):
  """
  Wrap a gym-retro environment and make it use discrete
  actions for the Sonic game.
  """
  def __init__(self, env):
    super(ActionsDiscretizer, self).__init__(env)
    buttons = ["B", "A", "MODE", "START", "UP", "DOWN", "LEFT", "RIGHT", "C", "Y", "X", "Z"]
    #actions = [['LEFT'], ['RIGHT'], ['LEFT', 'DOWN'], ['RIGHT', 'DOWN'], ['DOWN'],
              #['DOWN', 'B'], ['B']]
    actions = [['RIGHT'], ['RIGHT','C'], ['C']]
    self._actions = []

    """
    What we do in this loop:
    For each action in actions
      - Create an array of 12 False (12 = nb of buttons)
      For each button in action: (for instance ['LEFT']) we need to make that left button index = True
          - Then the button index = LEFT = True

      In fact at the end we will have an array where each array is an action and each elements True of this array
      are the buttons clicked.
    """
    for action in actions:
      arr = np.array([False] * 12)
      for button in action:
        arr[buttons.index(button)] = True
      self._actions.append(arr)
    self.action_space = gym.spaces.Discrete(len(self._actions))

  def action(self, a): # pylint: disable=W0221
    return self._actions[a].copy()

class RewardScaler(gym.RewardWrapper):
  """
  Bring rewards to a reasonable scale for PPO.
  This is incredibly important and effects performance
  drastically.
  """
  def reward(self, reward):
    return reward*0.01

class AllowBacktracking(gym.Wrapper):
  """
  Use deltas in max(X) as the reward, rather than deltas
  in X. This way, agents are not discouraged too heavily
  from exploring backwards if there is no way to advance
  head-on in the level.
  """
  def __init__(self, env):
    super(AllowBacktracking, self).__init__(env)
    self._cur_x = 0
    self._max_x = 0
    self.lives = 2

  def reset(self, **kwargs): # pylint: disable=E0202
    self._cur_x = 0
    self._max_x = 0
    return self.env.reset(**kwargs)

  def step(self, action): # pylint: disable=E0202
    obs, rew, done, info = self.env.step(action)
    #self._cur_x += rew
    #rew = max(0, self._cur_x - self._max_x)
    #self._max_x = max(self._max_x, self._cur_x)
    if(info["lives"] < self.lives or done == True):
      self.lives = info["lives"]
      rew = -2
    return obs, rew, done, info


def make_env(env_idx): 
  """
  Create an environment with some standard wrappers.
  """
  dicts = [{'game': 'SuperMarioBros-Nes', 'state': 'Level1-1'},
          {'game': 'SuperMarioBros-Nes', 'state': 'Level2-1'},
          {'game': 'SuperMarioBros-Nes', 'state': 'Level3-1'},
          {'game': 'SuperMarioBros-Nes', 'state': 'Level4-1'},
          {'game': 'SuperMarioBros-Nes', 'state': 'Level5-1'},
          {'game': 'SuperMarioBros-Nes', 'state': 'Level6-1'},
          {'game': 'SuperMarioBros-Nes', 'state': 'Level7-1'},
          {'game': 'SuperMarioBros-Nes', 'state': 'Level8-1'}]
      
  # Make the environment
  print(dicts[env_idx]['game'], dicts[env_idx]['state'], flush=True)
  #record_path = "./records/" + dicts[env_idx]['state']
  env = make_retro(game=dicts[env_idx]['game'], state=dicts[env_idx]['state'], record='.')

  # Build the actions array, 
  env = ActionsDiscretizer(env)

  # Scale the rewards
  env = RewardScaler(env)

  # PreprocessFrame
  env = PreprocessFrame(env)

  # Stack 4 frames
  env = FrameStack(env, 4)

  # Allow back tracking that helps agents are not discouraged too heavily
  # from exploring backwards if there is no way to advance
  # head-on in the level.
  env = AllowBacktracking(env)

  return env

def make_train_0():
  return make_env(0)

def make_train_1():
  return make_env(1)

def make_train_2():
  return make_env(2)

def make_train_3():
  return make_env(3)

def make_train_4():
  return make_env(4)

def make_train_5():
  return make_env(5)

def make_train_6():
  return make_env(6)

def make_train_7():
  return make_env(7)


In [ ]:
# Convolution layer
def conv_layer(inputs, filters, kernel_size, strides, gain=1.0):
    return tf.layers.conv2d(inputs=inputs, filters=filters, kernel_size=kernel_size,strides=(strides, strides),
                            activation=tf.nn.relu, kernel_initializer=tf.orthogonal_initializer(gain=gain))

# Fully connected layer
def fc_layer(inputs, units, activation_fn=tf.nn.relu, gain=1.0):
    return tf.layers.dense(inputs=inputs, units=units, activation=activation_fn,
                           kernel_initializer=tf.orthogonal_initializer(gain))

class A2CPolicy(object):
  """
  This object creates the A2C Network architecture
  """
  def __init__(self, sess, ob_space, action_space, nbatch, nsteps, reuse = False):
    # This will use to initialize our kernels
    gain = np.sqrt(2)

    # Based on the action space, will select what probability distribution type
    # we will use to distribute action in our stochastic policy (in our case DiagGaussianPdType
    # aka Diagonal Gaussian, 3D normal distribution
    self.pdtype = make_pdtype(action_space)

    height, weight, channel = ob_space.shape
    ob_shape = (height, weight, channel)

    # Create the input placeholder
    inputs_ = tf.placeholder(tf.float32, [None, *ob_shape], name="input")

    # Normalize the images
    scaled_images = tf.cast(inputs_, tf.float32) / 255.

    """
    Build the model
    3 CNN for spatial dependencies
    Temporal dependencies is handle by stacking frames
    (Something funny nobody use LSTM in OpenAI Retro contest)
    1 common FC
    1 FC for policy
    1 FC for value
    """
    with tf.variable_scope("model", reuse = reuse):
      conv1 = conv_layer(scaled_images, 32, 8, 4, gain)
      conv2 = conv_layer(conv1, 64, 4, 2, gain)
      conv3 = conv_layer(conv2, 64, 3, 1, gain)
      flatten1 = tf.layers.flatten(conv3)
      fc_common = fc_layer(flatten1, 512, gain=gain)

      # This build a fc connected layer that returns a probability distribution
      # over actions (self.pd) and our pi logits (self.pi).
      self.pd, self.pi = self.pdtype.pdfromlatent(fc_common, init_scale=0.01)
      
      # Calculate the v(s)
      vf = fc_layer(fc_common, 1, activation_fn=None)[:, 0]

    self.initial_state = None

    # Take an action in the action distribution (remember we are in a situation
    # of stochastic policy so we don't always take the action with the highest probability
    # for instance if we have 2 actions 0.7 and 0.3 we have 30% chance to take the second)
    a0 = self.pd.sample()

    # Function use to take a step returns action to take and V(s)
    def step(state_in, *_args, **_kwargs):
      action, value = sess.run([a0, vf], {inputs_: state_in})
      #print("step", action)
      return action, value

    # Function that calculates only the V(s)
    def value(state_in, *_args, **_kwargs):
      return sess.run(vf, {inputs_: state_in})

    # Function that output only the action to take
    def select_action(state_in, *_args, **_kwargs):
      return sess.run(a0, {inputs_: state_in})

    self.inputs_ = inputs_
    self.vf = vf
    self.step = step
    self.value = value
    self.select_action = select_action


In [ ]:
# Get the variables
def find_trainable_variables(key):
  with tf.variable_scope(key):
    return tf.trainable_variables()

# Make directory
def make_path(f):
  # exist_ok: if the folder already exist makes no exception error
  return os.makedirs(f, exist_ok=True)

def discount_with_dones(rewards, dones, gamma):
  discounted = []
  r = 0
  for reward, done in zip(rewards[::-1], dones[::-1]):
    r = reward + gamma*r*(1.-done) # fixed off by one bug
    discounted.append(r)
  return discounted[::-1]

In [ ]:
class Model(object):
  """
  We use this object to :
  __init__:
  - Creates the step_model
  - Creates the train_model

  train():
  - Make the training part (feedforward and retropropagation of gradients)

  save/load():
  - Save load the model
  """
  def __init__(self, policy, ob_space, action_space, nenvs, nsteps, ent_coef, vf_coef, max_grad_norm):
    sess = tf.get_default_session()

    # Here we create the placeholders
    actions_ = tf.placeholder(tf.int32, [None], name="actions_")
    advantages_ = tf.placeholder(tf.float32, [None], name="advantages_")
    rewards_ = tf.placeholder(tf.float32, [None], name="rewards_")
    lr_ = tf.placeholder(tf.float32, name="learning_rate_")

    # Here we create our two models:
    # Step_model that is used for sampling
    step_model = policy(sess, ob_space, action_space, nenvs, 1, reuse=False)

    # Train model for training
    train_model = policy(sess, ob_space, action_space, nenvs*nsteps, nsteps, reuse=True)

    """
    Calculate the loss
    Total loss = Policy gradient loss - entropy * entropy coefficient + Value coefficient * value loss
    """
    # Policy loss
    # Output -log(pi)
    neglogpac = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=train_model.pi, labels=actions_)

    # 1/n * sum A(si,ai) * -logpi(ai|si)
    pg_loss = tf.reduce_mean(advantages_ * neglogpac)

    # Value loss 1/2 SUM [R - V(s)]^2   
    vf_loss = tf.reduce_mean(tf.square(train_model.vf-rewards_))

    # Entropy is used to improve exploration by limiting the premature convergence to suboptimal policy.
    entropy = tf.reduce_mean(train_model.pd.entropy())

    loss = pg_loss - entropy * ent_coef + vf_loss * vf_coef
    print(pg_loss)
    print(loss)
    
    # Update parameters using loss
    # 1. Get the model parameters
    params = find_trainable_variables("model")

    # 2. Calculate the gradients
    grads = tf.gradients(loss, params)
    if max_grad_norm is not None:
      # Clip the gradients (normalize)
      grads, grad_norm = tf.clip_by_global_norm(grads, max_grad_norm)
    grads = list(zip(grads, params))
    # zip aggregate each gradient with parameters associated
    # For instance zip(ABCD, xyza) => Ax, By, Cz, Da

    # 3. Build our trainer
    trainer = tf.train.RMSPropOptimizer(learning_rate=lr_, decay=0.99, epsilon=1e-5)

    # 4. Backpropagation
    _train = trainer.apply_gradients(grads)
    
    def train(states_in, actions, returns, values, lr):
      # Here we calculate advantage A(s,a) = R + yV(s') - V(s)
      # Returns = R + yV(s')
      advantages = returns - values

      # We create the feed dictionary
      td_map = {train_model.inputs_: states_in, 
                actions_: actions,
                advantages_: advantages, # Use to calculate our policy loss
                rewards_: returns, # Use as a bootstrap for real value
                lr_: lr}

      policy_loss, value_loss, policy_entropy, _= sess.run([pg_loss, vf_loss, entropy, _train], td_map)

      return policy_loss, value_loss, policy_entropy

    def save(save_path):
      """
      Save the model
      """
      saver = tf.train.Saver()
      saver.save(sess, save_path)

    def load(load_path):
      """
      Load the model
      """
      saver = tf.train.Saver()
      print('Loading ' + load_path)
      saver.restore(sess, load_path)

    self.train = train
    self.train_model = train_model
    self.step_model = step_model
    self.step = step_model.step
    self.value = step_model.value
    self.initial_state = step_model.initial_state
    self.save = save
    self.load = load
    tf.global_variables_initializer().run(session=sess)

class Runner(AbstractEnvRunner):
  """
  We use this object to make a mini batch of experiences
  __init__:
  - Initialize the runner

  run():
  - Make a mini batch
  """
  def __init__(self, env, model, nsteps, total_timesteps, gamma, lam):    
    super().__init__(env = env, model = model, nsteps = nsteps)
    
    # Discount rate
    self.gamma = gamma

    # Lambda used in GAE (General Advantage Estimation)
    self.lam = lam

    # Total timesteps taken
    self.total_timesteps = total_timesteps    

  def run(self):
    # Here, we init the lists that will contain the mb of experiences
    mb_obs, mb_actions, mb_rewards, mb_values, mb_dones = [],[],[],[],[]

    # For n in range number of steps
    for n in range(self.nsteps):
      # Given observations, take action and value (V(s))
      # We already have self.obs because AbstractEnvRunner run self.obs[:] = env.reset()
      actions, values = self.model.step(self.obs, self.dones)

      #print("actions runner runner", actions)

      # Append the observations into the mb
      mb_obs.append(np.copy(self.obs)) #obs len nenvs (1 step per env)

      # Append the actions taken into the mb
      mb_actions.append(actions)

      # Append the values calculated into the mb
      mb_values.append(values)

      # Append the dones situations into the mb
      mb_dones.append(self.dones)

      # Take actions in env and look the results
      self.obs[:], rewards, self.dones, _ = self.env.step(actions)

      mb_rewards.append(rewards)

    #batch of steps to batch of rollouts
    mb_obs = np.asarray(mb_obs, dtype=np.uint8)
    mb_rewards = np.asarray(mb_rewards, dtype=np.float32)
    mb_actions = np.asarray(mb_actions, dtype=np.int32)
    mb_values = np.asarray(mb_values, dtype=np.float32)
    mb_dones = np.asarray(mb_dones, dtype=np.bool)
    last_values = self.model.value(self.obs)


    ### GENERALIZED ADVANTAGE ESTIMATION
    # discount/bootstrap off value fn
    # We create mb_returns and mb_advantages
    # mb_returns will contain Advantage + value
    mb_returns = np.zeros_like(mb_rewards)
    mb_advantages = np.zeros_like(mb_rewards)

    lastgaelam = 0

    # From last step to first step
    for t in reversed(range(self.nsteps)):
      # If t == before last step
      if t == self.nsteps - 1:
        # If a state is done, nextnonterminal = 0
        # In fact nextnonterminal allows us to do that logic

        #if done (so nextnonterminal = 0):
        #    delta = R - V(s) (because self.gamma * nextvalues * nextnonterminal = 0)
        # else (not done)
            #delta = R + gamma * V(st+1)
        nextnonterminal = 1.0 - self.dones

        # V(t+1)
        nextvalues = last_values
      else:
        nextnonterminal = 1.0 - mb_dones[t+1]

        nextvalues = mb_values[t+1]

      # Delta = R(st) + gamma * V(t+1) * nextnonterminal  - V(st)
      delta = mb_rewards[t] + self.gamma * nextvalues * nextnonterminal - mb_values[t]

      # Advantage = delta + gamma *  λ (lambda) * nextnonterminal  * lastgaelam
      mb_advantages[t] = lastgaelam = delta + self.gamma * self.lam * nextnonterminal * lastgaelam

    # Returns
    mb_returns = mb_advantages + mb_values

    return map(sf01, (mb_obs, mb_actions, mb_returns, mb_values))

def sf01(arr):
  """
  swap and then flatten axes 0 and 1
  """
  s = arr.shape
  return arr.swapaxes(0, 1).reshape(s[0] * s[1], *s[2:])

def learn(policy, env, nsteps, total_timesteps, gamma, lam, vf_coef, ent_coef, lr, max_grad_norm, log_interval):
    
  noptepochs = 4
  nminibatches = 8

  # Get the number of env
  nenvs = env.num_envs

  # Get state_space and action_space
  ob_space = env.observation_space
  ac_space = env.action_space

  # Calculate the batch_size
  batch_size = nenvs * nsteps # For instance if we take 5 steps and we have 5 environments batch_size = 25

  batch_train_size = batch_size // nminibatches

  assert batch_size % nminibatches == 0
  
  # Instantiate the model object (that creates step_model and train_model)
  model = Model(policy=policy, ob_space=ob_space, action_space=ac_space, nenvs=nenvs, nsteps=nsteps, 
                ent_coef=ent_coef, vf_coef=vf_coef, max_grad_norm=max_grad_norm)

  # Load the model
  # If you want to continue training
  if load:
    load_path = "model.ckpt"
    model.load(load_path)
  
  # Instantiate the runner object
  runner = Runner(env, model, nsteps=nsteps, total_timesteps=total_timesteps, gamma=gamma, lam=lam)
  
  # Index of each element of batch_size
  # Create the indices array
  indices = np.arange(batch_size)

  # Start total timer
  tfirststart = time.time()
  for update in range(1, total_timesteps//batch_size+1):
    # Start timer
    tstart = time.time()

    # Get minibatch
    obs, actions, returns, values = runner.run()

    # Here what we're going to do is for each minibatch calculate the loss and append it.
    mb_losses = []
    total_batches_train = 0

    for _ in range(noptepochs):
      # Randomize the indexes
      np.random.shuffle(indices)

      # 0 to batch_size with batch_train_size step
      for start in range(0, batch_size, batch_train_size):
        end = start + batch_train_size
        mbinds = indices[start:end]
        slices = (arr[mbinds] for arr in (obs, actions, returns, values))
        mb_losses.append(model.train(*slices, lr))

    # Feedforward --> get losses --> update
    lossvalues = np.mean(mb_losses, axis=0)

    # End timer
    tnow = time.time()

    # Calculate the fps (frame per second)
    fps = int(batch_size / (tnow - tstart))
    print(update)
    if update % log_interval == 0 or update == 1:
      """
      Computes fraction of variance that ypred explains about y.
      Returns 1 - Var[y-ypred] / Var[y]
      interpretation:
      ev=0  =>  might as well have predicted zero
      ev=1  =>  perfect prediction
      ev<0  =>  worse than just predicting zero
      """
      ev = explained_variance(values, returns)
      logger.record_tabular("nupdates", update)
      logger.record_tabular("total_timesteps", update*batch_size)
      logger.record_tabular("fps", fps)
      logger.record_tabular("policy_loss", float(lossvalues[0]))
      logger.record_tabular("policy_entropy", float(lossvalues[2]))
      logger.record_tabular("value_loss", float(lossvalues[1]))
      logger.record_tabular("explained_variance", float(ev))
      logger.record_tabular("time elapsed", float(tnow - tfirststart))
      logger.dump_tabular()

      savepath = "model.ckpt"
      model.save(savepath)
      print('Saving to', savepath)

  env.close()

def play(policy, env):
  # Get state_space and action_space
  ob_space = env.observation_space
  ac_space = env.action_space
  
  # Instantiate the model object (that creates step_model and train_model)
  model = Model(policy=policy, ob_space=ob_space, action_space=ac_space,
                nenvs=1, nsteps=1, ent_coef=0, vf_coef=0, max_grad_norm=0)

  # Load the model
  load_path = "model.ckpt"
  model.load(load_path)    

  # Play
  obs = env.reset()  
  score = 0
  done = False
  sequence_of_actions = []

  while done == False:
    # Get the action
    actions, values = model.step(obs)

    # Take actions in env and look the results
    obs, rewards, done, _ = env.step(actions)
    sequence_of_actions.append(actions)
    score += rewards

  print("Score ", score)
  env.close()
  return sequence_of_actions


In [ ]:
load = False
def main():
  config = tf.ConfigProto()

  # Avoid warning message errors
  os.environ["CUDA_VISIBLE_DEVICES"]="0"

  # Allowing GPU memory growth
  config.gpu_options.allow_growth = True

  tf.reset_default_graph()

  with tf.Session(config=config):
    learn(policy=A2CPolicy,
    env= SubprocVecEnv([make_train_0, make_train_1, make_train_2, make_train_3, make_train_4, make_train_5, make_train_6, make_train_7]),
    nsteps=2048, # Steps per environment
    total_timesteps=163840, gamma=0.99, lam = 0.95,
    vf_coef=0.5, ent_coef=0.01, lr = 2e-4, max_grad_norm = 0.5, log_interval = 5)

if __name__ == '__main__':
    main()


In [ ]:
files.download("checkpoint")
files.download("model.ckpt.data-00000-of-00001")
files.download("model.ckpt.index")
files.download("model.ckpt.meta")

In [ ]:
from baselines.common.vec_env.dummy_vec_env import DummyVecEnv

global sequence_of_actions
config = tf.ConfigProto()

# Avoid warning message errors
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Allowing GPU memory growth
config.gpu_options.allow_growth = True
tf.reset_default_graph()
with tf.Session(config=config):
  sequence_of_actions = play(policy=A2CPolicy, env= SubprocVecEnv([make_train_4]))


In [ ]:
env = make_train_4()
env.reset()

for i in range(len(sequence_of_actions)):
  for i in range(4):
    observation, reward, done, info = env.step(sequence_of_actions[i][0])
    if(done == True):
      break

env.close()

In [ ]:
!python3 -m retro.scripts.playback_movie /content/SuperMarioBros-Nes-Level5-1-000000.bk2